In [0]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.model_selection import KFold
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation,Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint

In [112]:
input_set_labeled = np.genfromtxt ('training.csv', delimiter=",")
input_set=input_set_labeled[1:input_set_labeled.shape[0],:]
input_data=input_set[:,1:input_set.shape[1]]
target_data = input_set[:,0]
target_data=target_data.reshape(target_data.shape[0],1)
print(input_data.shape)
print(target_data.shape)
print(input_set.shape[1])
print(input_set.shape[0])

(21000, 784)
(21000, 1)
785
21000


In [0]:
#input_data=tf.transpose(input_data)
# sess = tf.Session()
# new_input_data=tf.concat([input_data,target_data],1)
# new_input_data=tf.random_shuffle(new_input_data)
# init_var = tf.global_variables_initializer()
# sess.run(init_var)
# print(new_input_data.shape)
# new_input_data=sess.run(new_input_data)

In [0]:
# input_data=new_input_data[:,0:len(new_input_data[0])-5]
# target_data=new_input_data[:,len(new_input_data[0])-4:len(new_input_data[0])-1]
# print(input_data.shape)
# print(input_data.shape)
# input_data_trans = tf.transpose(input_data)
# target_data_trans = tf.transpose(target_data)
#input_data_min=input_data[:,0:len(input_data[0])]
# print(target_data_trans.shape)

In [115]:
sess = tf.Session()
init_var = tf.global_variables_initializer()
sess.run(init_var)
indices1 = tf.cast(target_data, tf.int32)
one_hot_vecs1 = tf.one_hot(indices1,np.max(sess.run(indices1))+1)
new_output=sess.run(one_hot_vecs1)
new_output=new_output.reshape(21000,10)
print(new_output)


[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]]


In [116]:
input_train,input_test, target_train, target_test =train_test_split(input_data_min,new_output,test_size=0.2, random_state=10)
print(input_train.shape)
print(input_test.shape)
print(target_train.shape)
print(target_test.shape)

(16800, 784)
(4200, 784)
(16800, 10)
(4200, 10)


In [117]:
input_train_map=input_train.reshape(-1,28,28,1)
input_test_map=input_test.reshape(-1,28,28,1)
print(input_train_map.shape)
print(input_test_map.shape)

(16800, 28, 28, 1)
(4200, 28, 28, 1)


In [0]:
model = Sequential()
#32 core，with size 3*3
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
#2*2 max pool
model.add(MaxPooling2D(pool_size=(2, 2)))
#drop out layor
#model.add(Dropout(0.25))
#64 core，with size 3*3
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
#2*2 max pool
model.add(MaxPooling2D(pool_size=(2, 2)))
#drop out layor
#model.add(Dropout(0.25))
#flatten layor
model.add(Flatten())
#full connected
model.add(Dense(512, activation='relu'))
#model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))


#sorting
model.add(Dense(10, activation='softmax'))

sgd = SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd)



In [125]:
model.fit(input_train_map, target_train, batch_size=96, epochs=100)



Epoch 1/100
16800/16800 [==============================] - 2s 117us/step - loss: 0.0517
Epoch 2/100
16800/16800 [==============================] - 2s 115us/step - loss: 0.0466
Epoch 3/100
16800/16800 [==============================] - 2s 116us/step - loss: 0.0398
Epoch 4/100
16800/16800 [==============================] - 2s 115us/step - loss: 0.0385
Epoch 5/100
16800/16800 [==============================] - 2s 115us/step - loss: 0.0332
Epoch 6/100
16800/16800 [==============================] - 2s 115us/step - loss: 0.0328
Epoch 7/100
16800/16800 [==============================] - 2s 115us/step - loss: 0.0294
Epoch 8/100
16800/16800 [==============================] - 2s 115us/step - loss: 0.0261
Epoch 9/100
16800/16800 [==============================] - 2s 116us/step - loss: 0.0286
Epoch 10/100
16800/16800 [==============================] - 2s 115us/step - loss: 0.0272
Epoch 11/100
16800/16800 [==============================] - 2s 115us/step - loss: 0.0234
Epoch 12/100
16800/16800 [====

In [126]:
score = model.evaluate(input_test_map, target_test, batch_size=8)
print(score)
# plot_model(model, to_file='modelcnn.png',show_shapes=True)


4200/4200 [==============================] - 1s 313us/step
0.07178178971240137


In [0]:
# input_data_test_finish = np.genfromtxt('data_test.csv', delimiter=",")
# input_data_min=input_data_test_finish[0:len(input_data_test_finish)-1]
# input_data_map=input_data_min.reshape(-1,640,480,1)
# predict_test=model.predict(input_data_map)
# print(predict_test)
# predict_test_array=predict_test.tolist()
# index=predict_test_array[0].index(max(predict_test_array[0]))
# if index==0:
#   print("I saw a rectangle")
# elif index==1:
#   print("I saw a gear")
# else:
#   print("It is not gear nor rectangle")

In [0]:
model.save("CNN.model")

In [0]:
model.save('my_model.h5')

In [0]:
model.save_weights('my_checkpoint')